<a href="https://colab.research.google.com/github/PinmanHuang/MLG/blob/master/DrBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

# Upload and Read Data



In [4]:
# upload file into colab
from google.colab import files
uploaded = files.upload()

Saving 0.txt to 0 (1).txt


In [0]:
# read upload data
import io
from numpy import genfromtxt
my_data = genfromtxt(io.BytesIO(uploaded['0.txt']), delimiter='\t')

# Initialization

##### Input

*   \\( G=(V, E) \\)
*   \\( X_v = \left[ d_v, 1, 1 \right] \\)
*   \\( h_v^{\left ( 0\right )} = X_v \\)




In [0]:
import networkx as nx

# calculate node degree
def cal_degree(graph): 
    G = nx.Graph()
    for val in graph:
        G.add_edge(val[0], val[1])

    return G.degree(), G

# create X = hv(0)
def create_features(graph):
    d, G = cal_degree(graph)
    X_ary = []
    for i in range(len(d)):
        X_ary.append([d[i], 1, 1])
    X = torch.tensor(X_ary)
    return G, d, X



In [0]:
G, d, X = create_features(my_data)
node_num = len(X)

# DrBC Network





In [0]:
class Net(torch.nn.Module):
    def __init__(self, D_in = 3, H = 2, num_units = 128):
        super(Net, self).__init__()

        self.D_in = D_in
        self.H = H
        self.num_units = num_units

        self.fc1 = torch.nn.Linear(self.D_in, self.H, bias=False)
        self.gru1 = torch.nn.GRUCell(self.num_units, self.num_units, bias=False)
        self.gru2 = torch.nn.GRUCell(self.num_units, self.num_units, bias=False)
        self.fc2 = torch.nn.Linear(self.num_units, int(self.num_units / 2), bias=False)
        self.fc3 = torch.nn.Linear(int(self.num_units / 2), 1, bias=False)

        # matrix of neighborhood
        self.neighbor = torch.zeros([node_num, node_num], dtype=torch.float32)
        e = [e for e in G.edges]
        for node in e[:5]:
            neighbor[int(node[0]), int(node[1])] = neighbor[int(node[1]), int(node[0])] = 1

        # matrix of aggregation weight
        self.aggregation_w = torch.zeros([node_num, node_num], dtype=torch.float32)
        for i in range(node_num):
            di = float(d[i])
            if i%500 == 0:
                print("calculating aggregation weights: {}/{}".format(i, node_num))
            for j in range (i):
                dj = float(d[j])
                self.aggregation_w[i, j] = self.aggregation_w[j, i] = 1/(((dj+1)**0.5) * ((di+1)**0.5))                


    def forward(self, x):
        # Encoding
        # ======================================================================
        # Initial Feature
        h0 = x
        h1 = self.fc1(h0).clamp(min=0)
        h1 = torch.nn.functional.normalize(h1)

        # 1st layer
        ##########################
        # Neighborhood Aggregation
        h2n = torch.mm(self.neighbor * self.aggregation_w, h1)
        # Combine Function
        h2 = self.gru1(h1, h2n)
        h2 = torch.nn.functional.normalize(h2)

        # 2nd layer
        ##########################
        # Neighborhood Aggregation
        h3n = torch.mm(self.neighbor * self.aggregation_w, h2)
        # Combine Function
        h3 = self.gru2(h2, h3n)
        h3 = torch.nn.functional.normalize(h3)

        # Layer Aggregation
        z = torch.max(torch.max(h1, h2), h3)

        # Decoder
        # ======================================================================
        y = self.fc2(z).clamp(min=0)
        y = self.fc3(y)

        return y

# Training

In [99]:
model = Net()
print(model)

calculating aggregation weights: 0/5000
calculating aggregation weights: 500/5000
calculating aggregation weights: 1000/5000
calculating aggregation weights: 1500/5000
calculating aggregation weights: 2000/5000
calculating aggregation weights: 2500/5000
calculating aggregation weights: 3000/5000
calculating aggregation weights: 3500/5000
calculating aggregation weights: 4000/5000
calculating aggregation weights: 4500/5000
Net(
  (fc1): Linear(in_features=3, out_features=2, bias=False)
  (gru1): GRUCell(128, 128, bias=False)
  (gru2): GRUCell(128, 128, bias=False)
  (fc2): Linear(in_features=128, out_features=64, bias=False)
  (fc3): Linear(in_features=64, out_features=1, bias=False)
)
